In [ ]:
from models_DQ_showoff import *
from IA_Trafficlight import *

In [ ]:
print("Hello World!")

Hello World!


In [ ]:
one_hot = True
write = False
board = Board(width = 31, height = 31, spawn_rate=5, max_spawn_batch=6, one_hot = one_hot, write=write,targetSuccessfulTrips=200)

In [ ]:
IA = TrafficLightIA(board,epsilon=0.1)

In [ ]:

state = IA.env.reset()
state = torch.tensor(state).to(IA.device)
# We reshape the state to be a 1x(11x31x31) tensor
state = state.view(1, 11*31*31)
print("-"*50)
print("Episode: ", 0)
print("-"*50)
state = state.float()
score = 0
done = False
steps = 0
while not done:
    action,wasRandom = IA.getMoves(state, IA.get_epsilon(0))
    # Give time so the cars can move
    for _ in range(10): 
        next_state, reward, done, _ = IA.env.step(action)
    next_state = IA.tensor(next_state).to(IA.device)
    next_state = next_state.view(1,11*31*31)
    next_state = next_state.float()
    state = next_state
    score += reward
    steps += 1
    if steps > 200:
        break

In [ ]:
%%capture
paleta = np.array(
    [
    # 0 -> highway
    [30 , 30 , 30 ],  
    # 1 -> up traffic light, red
    [255, 0  , 0  ],
    # 2 -> down traffic light, green
    [0  , 255, 0  ],
    # 3 -> road, dark green
    [0  , 60, 0  ],
    # 4 -> car looking up, purple
    [255, 0  , 255],
    # 5 -> car looking down, orange
    [255, 165, 0  ],
    # 6 -> car looking left, pink
    [255, 192, 203],
    # 7 -> car looking right, cyan
    [0  , 255, 255],
    # 8 -> lane
    [255, 255, 255]
    ])
all_grid = board.datacollector.get_model_vars_dataframe()
pinta_me = lambda matriz: paleta[matriz.astype(int)]
fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(pinta_me(all_grid.iloc[0][0]))
def animate(i):
    patch.set_data(pinta_me(all_grid.iloc[i][0]))
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
# Success rate
if board.successful_trips:
    print("Success rate:", "{:.2f}".format(board.successful_trips / (board.successful_trips + board.crashes)*100)+"%")
    print(f"Crashes: {int(board.crashes)}")
    print(f"Successful trips: {board.successful_trips}")
    print(f"Time stuck: {board.time_stuck}")
    print("Reward:", board.rewardFunction())
else:
    print("No successful trips")

In [ ]:
anim